# x402 Facilitator for Optimism 

In this notebook, we test the x402 facilitator for Optimism network.

**Deployed on Scaleway Functions:**
- Verify: `https://x402facilitatorjccmtmdr-verify.functions.fnc.fr-par.scw.cloud`
- Settle: `https://x402facilitatorjccmtmdr-settle.functions.fnc.fr-par.scw.cloud`
- Supported: `https://x402facilitatorjccmtmdr-supported.functions.fnc.fr-par.scw.cloud`

In [38]:
import requests
import json
from datetime import datetime

# Facilitator endpoint - Single function with path-based routing
# FACILITATOR_URL = "https://x402facilitatorjccmtmdr-facilitator.functions.fnc.fr-par.scw.cloud"
FACILITATOR_URL = "https://facilitator.fretchen.eu"
# FACILITATOR_URL = "http://localhost:8080"

VERIFY_URL = f"{FACILITATOR_URL}/verify"
SETTLE_URL = f"{FACILITATOR_URL}/settle"
SUPPORTED_URL = f"{FACILITATOR_URL}/supported"

# For local testing, uncomment this:
# FACILITATOR_URL = "http://localhost:8080"
# VERIFY_URL = f"{FACILITATOR_URL}/verify"
# SETTLE_URL = f"{FACILITATOR_URL}/settle"
# SUPPORTED_URL = f"{FACILITATOR_URL}/supported"

# After custom domain setup (recommended):
# FACILITATOR_URL = "https://facilitator.fretchen.eu"
# VERIFY_URL = f"{FACILITATOR_URL}/verify"
# SETTLE_URL = f"{FACILITATOR_URL}/settle"
# SUPPORTED_URL = f"{FACILITATOR_URL}/supported"

print("🚀 x402 Facilitator Endpoints (Single Function with Path Routing):")
print(f"  Base: {FACILITATOR_URL}")
print(f"  Verify: {VERIFY_URL}")
print(f"  Settle: {SETTLE_URL}")
print(f"  Supported: {SUPPORTED_URL}")

🚀 x402 Facilitator Endpoints (Single Function with Path Routing):
  Base: https://facilitator.fretchen.eu
  Verify: https://facilitator.fretchen.eu/verify
  Settle: https://facilitator.fretchen.eu/settle
  Supported: https://facilitator.fretchen.eu/supported


## Test /supported Endpoint

The `/supported` endpoint returns information about which networks, payment schemes, and tokens the facilitator supports.

**Note:** The facilitator now uses a single function with path-based routing:
- Single Scaleway Function deployment
- Path routing: `/verify`, `/settle`, `/supported`
- x402 standard compliant


## Service Fee Structure

The x402 Facilitator charges a **flat $0.01 fee per transaction** to cover:
- Gas costs for on-chain settlement (~$0.005 on Optimism)
- Infrastructure costs (hosting, monitoring)
- Nonce tracking and security

**Fee Comparison:**
- Traditional payment processors: 2.9% + $0.30 (Stripe), 2.99% + $0.49 (PayPal)
- x402 Facilitator: **Flat $0.01** regardless of transaction size

**How it works:**
1. User signs authorization for `payment_amount + $0.01` (e.g., $1.01 for a $1 payment)
2. Facilitator transfers `payment_amount` to recipient
3. Facilitator keeps the $0.01 fee to cover costs

This is much cheaper than traditional processors, especially for larger transactions!

In [39]:
# Test the /supported endpoint
response = requests.get(SUPPORTED_URL)

print(f"Status Code: {response.status_code}")
supported = response.json()
print(json.dumps(supported, indent=2))

print(f"\n✅ Supported Networks:")
for kind in supported['kinds']:
    print(f"  - {kind['network']} ({kind['scheme']} scheme)")
    if 'assets' in kind:
        for asset in kind['assets']:
            print(f"    • {asset['name']} ({asset['symbol']}): {asset['address']}")
            # Show fee information if available
            if 'fees' in asset:
                fees = asset['fees']
                print(f"      💰 Fees: {fees['feeModel']} model - ${fees['flatFeeUsd']} per transaction")
                print(f"         Minimum: ${fees['minTransactionUsd']} ({fees['minTransaction']} {fees['token']} units)")


Status Code: 200
{
  "kinds": [
    {
      "x402Version": 2,
      "scheme": "exact",
      "network": "eip155:10",
      "assets": [
        {
          "address": "0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85",
          "name": "USDC",
          "symbol": "USDC",
          "decimals": 6,
          "fees": {
            "token": "USDC",
            "chainId": 10,
            "decimals": 6,
            "feeModel": "flat",
            "flatFee": "10000",
            "flatFeeUsd": 0.01,
            "minTransaction": "100000",
            "minTransactionUsd": 0.1
          }
        }
      ]
    },
    {
      "x402Version": 2,
      "scheme": "exact",
      "network": "eip155:11155420",
      "assets": [
        {
          "address": "0x5fd84259d66Cd46123540766Be93DFE6D43130D7",
          "name": "USDC",
          "symbol": "USDC",
          "decimals": 6,
          "fees": {
            "token": "USDC",
            "chainId": 11155420,
            "decimals": 6,
            "feeModel"

In [ ]:
from eth_account import Account
from eth_account.messages import encode_typed_data
from eth_keys import keys
from web3 import Web3
import os
from dotenv import load_dotenv
import secrets

# Load environment variables
load_dotenv()
private_key = os.getenv('TEST_WALLET_PRIVATE_KEY')
pay_to_address = os.getenv('NFT_WALLET_PUBLIC_KEY')

# Create account from private key
account = Account.from_key(private_key)
from_address = Web3.to_checksum_address(account.address)
pay_to_address = Web3.to_checksum_address(pay_to_address)
print(f"Payer Address: {from_address}")
print(f"Recipient Address: {pay_to_address}")

# Network and token configuration
CHAIN_ID = 11155420  # Optimism Sepolia
USDC_ADDRESS = "0x5fd84259d66Cd46123540766Be93DFE6D43130D7"

# Payment configuration
payment_amount = "10000"  # $0.01 USDC (6 decimals) - amount for recipient
service_fee = "10000"       # $0.01 USDC (flat fee for facilitator)
total_amount = str(int(payment_amount) + int(service_fee))  # $0.02 USDC total

print(f"\n💰 Payment Details:")
print(f"  Payment to recipient: {payment_amount} ({int(payment_amount)/1e6:.2f} USDC)")
print(f"  Service fee: {service_fee} ({int(service_fee)/1e6:.2f} USDC)")
print(f"  Total authorization: {total_amount} ({int(total_amount)/1e6:.2f} USDC)")

# Create authorization data
valid_after = int(datetime.now().timestamp()) - 60
valid_before = int(datetime.now().timestamp()) + 3600
nonce = "0x" + secrets.token_hex(32)

# EIP-712 domain for USDC on Optimism Sepolia
# IMPORTANT: The contract's name() returns "USDC", not "USD Coin"!
domain_data = {
    "name": "USDC",
    "version": "2",us
    "chainId": CHAIN_ID,
    "verifyingContract": USDC_ADDRESS
}

# EIP-712 types for TransferWithAuthorization
types = {
    "EIP712Domain": [
        {"name": "name", "type": "string"},
        {"name": "version", "type": "string"},
        {"name": "chainId", "type": "uint256"},
        {"name": "verifyingContract", "type": "address"}
    ],
    "TransferWithAuthorization": [
        {"name": "from", "type": "address"},
        {"name": "to", "type": "address"},
        {"name": "value", "type": "uint256"},
        {"name": "validAfter", "type": "uint256"},
        {"name": "validBefore", "type": "uint256"},
        {"name": "nonce", "type": "bytes32"}
    ]
}

# Message data - IMPORTANT: Sign the TOTAL amount (payment + fee)
# The facilitator will transfer payment_amount to recipient and keep the fee
message_data = {
    "from": from_address,
    "to": pay_to_address,
    "value": int(total_amount),  # Sign total amount including fee
    "validAfter": int(valid_after),
    "validBefore": int(valid_before),
    "nonce": nonce
}

# Create the full EIP-712 message
full_message = {
    "types": types,
    "primaryType": "TransferWithAuthorization",
    "domain": domain_data,
    "message": message_data
}

# Encode to get the EIP-712 hash components
signable_message = encode_typed_data(full_message=full_message)

# Compute the FULL EIP-712 hash according to spec:
# keccak256("\x19\x01" || domainSeparator || hashStruct(message))
from eth_hash.auto import keccak
prefix = bytes.fromhex("1901")
domain_separator = signable_message.header  # This is hashStruct(domain)
message_hash = signable_message.body  # This is hashStruct(message)
full_eip712_hash = keccak(prefix + domain_separator + message_hash)

# Sign using eth_keys (this creates a pure EIP-712 signature)
private_key_bytes = bytes.fromhex(private_key.replace('0x', ''))
pk = keys.PrivateKey(private_key_bytes)
signature_obj = pk.sign_msg_hash(full_eip712_hash)

# Extract v, r, s components
v_raw, r, s = signature_obj.vrs

# Adjust v to be 27 or 28 (Ethereum standard)
if v_raw == 0:
    v = 27
elif v_raw == 1:
    v = 28
else:
    v = v_raw

# Combine into signature hex string: 0x + r + s + v
r_bytes = r.to_bytes(32, byteorder='big')
s_bytes = s.to_bytes(32, byteorder='big')
signature_hex = '0x' + r_bytes.hex() + s_bytes.hex() + format(v, '02x')

print(f"\n🔐 Signature Details:")
print(f"EIP-712 Hash: {full_eip712_hash.hex()}")
print(f"Signature: {signature_hex}")
print(f"Signature length: {len(signature_hex)} chars (should be 132 with 0x)")
print(f"Nonce: {nonce}")
print(f"\n✅ Authorized {int(total_amount)/1e6:.2f} USDC total (${int(payment_amount)/1e6:.2f} payment + ${int(service_fee)/1e6:.2f} fee)")


Payer Address: 0x553179556FC2A39e535D65b921e01fA995E79101
Recipient Address: 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C

💰 Payment Details:
  Payment to recipient: 1000000 (1.00 USDC)
  Service fee: 10000 (0.01 USDC)
  Total authorization: 1010000 (1.01 USDC)

🔐 Signature Details:
EIP-712 Hash: 48a811087144da948859c26bd70ab62db3f813220e514eaf39f4275767bcaa9b
Signature: 0x5d5b6cc767e35be7131138608877d56bdefb4e386457d09c96ac05d8065046e70482bed91a06ba4debc8f4296bfd5e676c87cbbcfb3cfbf2767e734d0e634c891b
Signature length: 132 chars (should be 132 with 0x)
Nonce: 0xd9cd5b7a9e1cc22174e934df6769906f03e789342b9320f512f20707455e6f7d

✅ Authorized 1.01 USDC total ($1.00 payment + $0.01 fee)


### Debug: Prüfe ob Nonce bereits verwendet wurde

Manchmal wurde eine Nonce bereits in einem früheren Test verwendet. Lass uns prüfen:

In [41]:
# Check if the current nonce was already used on-chain
print(f"Prüfe Nonce: {nonce}")
print(f"Payer: {from_address}")

# USDC contract ABI for authorizationState
usdc_abi = [
    {
        "inputs": [
            {"name": "authorizer", "type": "address"},
            {"name": "nonce", "type": "bytes32"}
        ],
        "name": "authorizationState",
        "outputs": [{"name": "", "type": "bool"}],
        "stateMutability": "view",
        "type": "function"
    }
]

# Connect to Optimism Sepolia
w3 = Web3(Web3.HTTPProvider('https://sepolia.optimism.io'))
usdc_contract = w3.eth.contract(address=USDC_ADDRESS, abi=usdc_abi)

# Check if nonce is used
nonce_used = usdc_contract.functions.authorizationState(from_address, nonce).call()

if nonce_used:
    print(f"\n❌ Diese Nonce wurde bereits verwendet!")
    print(f"Du musst die vorherige Zelle erneut ausführen, um eine neue Nonce zu generieren.")
else:
    print(f"\n✅ Diese Nonce ist noch verfügbar und kann verwendet werden.")

Prüfe Nonce: 0xd9cd5b7a9e1cc22174e934df6769906f03e789342b9320f512f20707455e6f7d
Payer: 0x553179556FC2A39e535D65b921e01fA995E79101



✅ Diese Nonce ist noch verfügbar und kann verwendet werden.


In [42]:
# Now create a valid x402 payment payload with our real signature
# IMPORTANT: The 'amount' in the request is the PAYMENT amount (without fee)
# But the signature authorizes the TOTAL amount (payment + fee)

real_payment_payload = {
    "x402Version": 2,
    "resource": {
        "url": "https://api.example.com/premium-data",
        "description": "Access to premium market data",
        "mimeType": "application/json"
    },
    "accepted": {
        "scheme": "exact",
        "network": "eip155:11155420",  # Optimism Sepolia testnet
        "amount": payment_amount,  # Payment amount WITHOUT fee (recipient gets this)
        "asset": USDC_ADDRESS,
        "payTo": pay_to_address,
        "maxTimeoutSeconds": 60,
        "extra": {
            "name": "USDC",
            "version": "2"
        }
    },
    "payload": {
        "signature": signature_hex,
        "authorization": {
            "from": from_address,
            "to": pay_to_address,
            "value": total_amount,  # Total amount WITH fee (what we signed)
            "validAfter": str(valid_after),
            "validBefore": str(valid_before),
            "nonce": nonce
        }
    }
}

# Create payment requirements
real_payment_requirements = {
    "scheme": "exact",
    "network": "eip155:11155420",
    "amount": payment_amount,  # Payment amount WITHOUT fee
    "asset": USDC_ADDRESS,
    "payTo": pay_to_address,
    "maxTimeoutSeconds": 60,
    "extra": {
        "name": "USDC",
        "version": "2"
    }
}

# Create verify request
real_verify_request = {
    "paymentPayload": real_payment_payload,
    "paymentRequirements": real_payment_requirements
}

print(f"\n📤 Request Summary:")
print(f"  Payment Amount (to recipient): ${int(payment_amount)/1e6:.2f} USDC")
print(f"  Authorized Amount (signed): ${int(total_amount)/1e6:.2f} USDC")
print(f"  Service Fee: ${int(service_fee)/1e6:.2f} USDC")
print(f"  Recipient: {pay_to_address}")

# Send to facilitator (deployed on Scaleway)
response = requests.post(
    VERIFY_URL,
    headers={"Content-Type": "application/json"},
    json=real_verify_request
)

result = response.json()
print(f"\n✅ Verify Response (Status {response.status_code}):")
print(json.dumps(result, indent=2))

if result.get('isValid'):
    print("\n🎉 Payment signature is valid and ready for settlement!")
    print(f"   Facilitator will transfer ${int(payment_amount)/1e6:.2f} to recipient")
    print(f"   Facilitator keeps ${int(service_fee)/1e6:.2f} as service fee")
else:
    print(f"\n❌ Validation failed: {result.get('invalidReason')}")



📤 Request Summary:
  Payment Amount (to recipient): $1.00 USDC
  Authorized Amount (signed): $1.01 USDC
  Service Fee: $0.01 USDC
  Recipient: 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C

✅ Verify Response (Status 200):
{
  "isValid": false,
  "invalidReason": "insufficient_funds",
  "payer": "0x553179556FC2A39e535D65b921e01fA995E79101"
}

❌ Validation failed: insufficient_funds

✅ Verify Response (Status 200):
{
  "isValid": false,
  "invalidReason": "insufficient_funds",
  "payer": "0x553179556FC2A39e535D65b921e01fA995E79101"
}

❌ Validation failed: insufficient_funds


## Settlement via Facilitator

Nach erfolgreicher Verifikation kann der Facilitator die Zahlung on-chain ausführen. Der `/settle` Endpoint:
1. Führt nochmals alle Verifikationen durch
2. Ruft `transferWithAuthorization` auf dem USDC-Contract auf
3. Gibt die Transaction Hash zurück

**Voraussetzungen für Settlement:**
- Der Payer braucht USDC auf Optimism Sepolia
- Der Facilitator braucht einen Private Key (`FACILITATOR_WALLET_PRIVATE_KEY` in `.env`) und etwas ETH für Gas
- Für diesen Test kannst du deinen eigenen Private Key verwenden (im echten Betrieb sollte der Facilitator einen separaten Wallet haben)

### Prüfe Facilitator-Wallet Balance

Bevor wir Settlement versuchen, prüfen wir ob der Facilitator genug ETH für Gas hat:

In [43]:
# Check facilitator wallet balance using public key from .env
facilitator_address = os.getenv('FACILITATOR_WALLET_PUBLIC_KEY')

if facilitator_address:
    facilitator_address = Web3.to_checksum_address(facilitator_address)
    
    # Connect to Optimism Sepolia to check balance
    w3 = Web3(Web3.HTTPProvider('https://sepolia.optimism.io'))
    
    if w3.is_connected():
        balance_wei = w3.eth.get_balance(facilitator_address)
        balance_eth = w3.from_wei(balance_wei, 'ether')
        
        print(f"Facilitator Wallet: {facilitator_address}")
        print(f"ETH Balance: {balance_eth} ETH")
        
        # Estimate gas cost (rough estimate: ~100k gas)
        estimated_gas = 100000
        gas_price = w3.eth.gas_price
        estimated_cost_wei = estimated_gas * gas_price
        estimated_cost_eth = w3.from_wei(estimated_cost_wei, 'ether')
        
        print(f"Geschätzte Gas-Kosten: ~{estimated_cost_eth:.6f} ETH")
        
        if balance_wei == 0:
            print(f"\n❌ FEHLER: Facilitator Wallet hat kein ETH!")
            print(f"Sende ETH an: {facilitator_address}")
            print(f"Bridge: https://app.optimism.io/bridge")
        elif balance_wei < estimated_cost_wei:
            print(f"\n⚠️  WARNUNG: Balance könnte zu niedrig für Gas sein")
            print(f"Empfohlen: Mindestens {estimated_cost_eth:.6f} ETH")
        else:
            print(f"\n✅ Ausreichend ETH für Settlement vorhanden")
    else:
        print("❌ Konnte nicht mit Optimism Sepolia verbinden")
else:
    print("❌ FACILITATOR_WALLET_PUBLIC_KEY nicht in .env konfiguriert")

Facilitator Wallet: 0x3F8d2Fb6fEA24E70155bC61471936F3c9C30c206
ETH Balance: 0.049999404554059615 ETH
Geschätzte Gas-Kosten: ~0.000000 ETH

✅ Ausreichend ETH für Settlement vorhanden


In [44]:
# Try to settle the payment (execute on-chain)
# Note: This will fail if the payer doesn't have USDC or if the facilitator wallet has no ETH for gas

print(f"\n💸 Attempting Settlement...")
print(f"  Payment: ${int(payment_amount)/1e6:.2f} USDC → {pay_to_address}")
print(f"  Fee: ${int(service_fee)/1e6:.2f} USDC → Facilitator")
print(f"  Total: ${int(total_amount)/1e6:.2f} USDC")

settle_response = requests.post(
    SETTLE_URL,
    headers={"Content-Type": "application/json"},
    json=real_verify_request  # Same request structure as verify
)

settle_result = settle_response.json()
print(f"\n📦 Settle Response (Status {settle_response.status_code}):")
print(json.dumps(settle_result, indent=2))

if settle_result.get('success'):
    tx_hash = settle_result.get('transactionHash')
    print(f"\n🎉 Settlement erfolgreich!")
    print(f"\n📝 Transaction Details:")
    print(f"   Hash: {tx_hash}")
    print(f"   Recipient received: ${int(payment_amount)/1e6:.2f} USDC")
    print(f"   Service fee: ${int(service_fee)/1e6:.2f} USDC")
    print(f"   Block Explorer: https://sepolia-optimism.etherscan.io/tx/{tx_hash}")
else:
    print(f"\n❌ Settlement fehlgeschlagen")
    error_reason = settle_result.get('errorReason', settle_result.get('invalidReason', 'unknown'))
    print(f"   Grund: {error_reason}")
    
    if 'insufficient' in error_reason.lower():
        print(f"\n💡 Tipp: Stelle sicher, dass:")
        print(f"   - Payer hat ${int(total_amount)/1e6:.2f} USDC auf Optimism Sepolia")
        print(f"   - Facilitator hat ETH für Gas")



💸 Attempting Settlement...
  Payment: $1.00 USDC → 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C
  Fee: $0.01 USDC → Facilitator
  Total: $1.01 USDC



📦 Settle Response (Status 200):
{
  "success": false,
  "errorReason": "insufficient_funds",
  "payer": "0x553179556FC2A39e535D65b921e01fA995E79101",
  "transaction": "",
  "network": "eip155:11155420"
}

❌ Settlement fehlgeschlagen
   Grund: insufficient_funds

💡 Tipp: Stelle sicher, dass:
   - Payer hat $1.01 USDC auf Optimism Sepolia
   - Facilitator hat ETH für Gas
